In [1]:
import numpy as np
import math
import pandas as pd
import nltk as nltk
import math
import operator
import ast
nltk.download('punkt')


[nltk_data] Downloading package punkt to /home/manorlf/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df = pd.read_csv("../../data/estadao_noticias_eleicao.csv")
df = df.fillna('')

In [3]:
# Feedback

feed_back = pd.read_csv('gabarito.csv')
feed_back




,str_busca,google,busca_binaria,tf,tfidf,bm25
0,segundo turno,"[1062, 1942, 2161, 2078, 2073]","[2048, 1, 2049, 2050, 4096]","[2744, 7, 2112, 7672, 2388]","[2744, 2112, 7672, 1235, 2388]","[2744, 2112, 7672, 2388, 2178]"
1,lava jato,"[616, 164, 1734, 163, 6716]","[3, 13, 15, 27, 6177]","[163, 353, 2807, 127, 359]","[163, 353, 2807, 127, 359]","[163, 353, 2807, 127, 359]"
2,projeto de lei,"[2853, 275, 978, 7092, 3171]","[3584, 6145, 8194, 8706, 6660]","[7, 3942, 7017, 1250, 6942]","[2232, 6461, 2853, 3171, 3942]","[2232, 6461, 3171, 2853, 3170]"
3,compra de voto,"[2200, 8615, 2265, 7746, 82]","[7424, 2178, 6531, 5122, 2311]","[3942, 7017, 5129, 2047, 748]","[7343, 7293, 6791, 3942, 2047]","[7343, 7293, 6791, 7329, 8615]"
4,ministério público,"[64, 6652, 164, 6550, 8615]","[8194, 7, 4104, 8201, 4109]","[6798, 8018, 6244, 6965, 6550]","[6798, 8018, 6244, 6965, 6550]","[6798, 8018, 6244, 6965, 6550]"


<h1>Vector Space Retrieval Model</h1>

In [5]:
INVERTED_INDEX = dict()

In [6]:
def search_term(term, data):
    '''
    Search term presence in data frame and calculate its term frequence. Also full fill the dictionary 
    with the result for future new searchs.
            
    ARGS:
        term: String with single word, that word is the term to be seach in the data frame.
        data: Data frame, where the terms will be search.
        
    RETURN:
        tuple (int n, list l): where n is the total of documents witch the term is presente at least once
                               l is a list of tuple (doc, tf), where doc is the notice id and tf is its 
                               term frequence.
    '''
    
    result = []
    cont = 0
    
    if (term in INVERTED_INDEX):
        result = INVERTED_INDEX[term]
    else:
        
        rows = data.shape[0]
                
        for doc in range(rows):
            title = (data.loc[doc, 'titulo']).lower()
            sub_title = (data.loc[doc, 'subTitulo']).lower()
            content = (data.loc[doc, 'conteudo']).lower()
                        
            tf = 0
            text = nltk.word_tokenize(title + ' ' + sub_title + ' ' + content)
            i = 0
            
            while (i < len(text)):
                if (text[i].lower() == term):
                    tf += 1
                    exist = True
                i += 1
            if (tf):
                id_notice = data.loc[doc, 'idNoticia']
                result.append((id_notice, tf))
                cont += 1
        INVERTED_INDEX[term] = (cont, result)
    
    return INVERTED_INDEX[term]

In [7]:

def intersection(query, data): 
    '''
    To get a dictionary with all notice id and its terms frequency in a list representing for all terms
    of the query.
        
    ARGS:
        query: String with the with words or terms to be seach
        data: Data frame, where the terms will be search.
        
    RETURN:
        {doc:l} doc is the notice id and l is a list with the tf (term frequency) of each term in the query
        
    '''
    q = query.split()
    dic_docs = dict()
    
    for term in q:
        term_set = search_term(term, data)
        for doc in term_set[1]:
            if (doc[0] in dic_docs):
                dic_docs[doc[0]].append(doc[1])
            else:
                dic_docs[doc[0]] = [doc[1]]
    return dic_docs
    
    

<h1>VSM Binary</h1>

In [8]:
def vsm_binary(query, data):
    '''
    Virtual Space Model: Binary
        Search for a specific query in a data frame with binary logic. 
        But here we use a intersection, all notice id needs to be in all docs simultaneously
        
    ARGS:
        query: String with the with words or terms to be seach
        data: Data frame, where the terms will be search.
        
    RETURN:
        List of tuple (doc, tf), where doc repersents the notice id and tf its term frequency
    
    '''
    q = query.split()
    dic_docs = intersection(query,data)
   
    result = []
    
    for doc in dic_docs.keys():
        if (len(dic_docs[doc]) == len(q)):
            result.append((doc, len(q)))
            
    return result    

<h1>VSM TF</h1>

In [9]:
def vsm_tf(query, data):
    '''
    Improved VSM with Term Frequency Weighting
        Search for a specific query in a data frame with TF logic. For each term in the query it will result,,
        a notice id with the sum of tf for each doc witch it's is present the term.
        But here we use a intersection, all notice id needs to be in all docs simultaneously.
        
    ARGS:
        query: String with the with words or terms to be seach
        data: Data frame, where the terms will be search.
        
    RETURN:
        List of tuple (doc, tf), where doc repersents the notice id and tf its term frequency
    
    '''
    q = query.split()
    dic_docs = intersection(query,data)
    
    result = []
    
    for doc in dic_docs.keys():
        if (len(dic_docs[doc]) == len(q)):
            result.append((doc, sum(dic_docs[doc])))
            
    return result

<h1>VSM TF-IDF</h1>

In [10]:
def vsm_tf_idf(query, data):
    '''
    Improvement of Vector Placement: Adding Inverse Document Frequency (IDF)
    IDF(W) = log[(M+1)/k]
        W: Word
        M: Total of Numbers in colection
        K: Total of documents containing  W (Doc Frequency )
        
    Trying to improve search using TF-IDF. For each term it will calculate IDF and multiply for tf, and then sum
    in order to improve Weighting.
    Here we use a intersection, all docs selected needs to have the term at least once.
        
    ARGS:
        query: String with the with words or terms to be seach.
        data: Data frame, where the terms will be search.
        
    RETURN:
        List of tuple (doc, tf), where doc repersents the notice id and tf-idf calc.
    
    '''
    q = query.split()
    dic_docs = intersection(query, data)
    
    
    result = []
    
    for doc in dic_docs.keys():
        tf_idf_sum = 0
        if (len(dic_docs[doc]) == len(q)):
            for i in range(len(q)):
                tf_idf_sum += dic_docs[doc][i] * idf(q[i], data)
            result.append((doc, tf_idf_sum))
            
    return result





def idf(term, data):
    '''
    Calculates IDF of a Word.
        IDF(W) = log[(M+1)/k]
            W: Word
            M: Total of Numbers in colection
            K: Total of documents containing  W (Doc Frequency )
    ARGS:
        term: Term or word to be calculated the idf.
        data: Data frame.
    
    RETURN:
        IDF for term.
    '''
    size = search_term(term, data)[0]
    total_docs = data.shape[0]
    calc = math.log((total_docs - 1)/ size)
    
    return calc

<h1>BM25</h1>

In [11]:
def vsm_bm25(query, data):
    '''
    BM25 Transformation 
        – has an upper bound
        – is robust and effective 
     
    ARGS:
        query: String with the with words or terms to be seach.
        data: Data frame, where the terms will be search.
    
    RETURN:
         List of tuple (doc, tf), where doc repersents the notice id and its BM25.
    '''
    q = query.split()
    dic_docs = intersection(query, data)
    
    result = []
    
    for doc in dic_docs.keys():
        bm25_sum = 0
        if (len(dic_docs[doc]) == len(q)):
            for i in range(len(q)):
                bm25_sum += bm25(dic_docs[doc][i])
            result.append((doc, bm25_sum))
            
    return result
    
    
def bm25(tf):
    '''
    Calculates the BM25 for a specific tf
    
    '''
    k = 5
    calc = ((k + 1) * tf)/(tf + k)
    return calc

<h1>Mean Average Precision (MAP)</h1>

In [12]:

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)result
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [13]:
def top_five(query, model, data):
    '''
    Given a specific model it will make a search for the docs witch the terms are presents.
    
    ARGS:
        query: String with the with words or terms to be seach.
        model: Specif model for the search, should be binay, tf, tf-idf or bm25
        data: Data frame, where the terms will be search. 
        
    RETURN:
        Top 5 documents for the search.
        
    '''
    result = []
    answer = []
    if (model == 'binary'):
        result = vsm_binary(query, data)
    elif (model == 'tf'):
        result = vsm_tf(query, data)
    elif (model == 'tf-idf'):
        result = vsm_tf_idf(query, data)
    else:
        result = vsm_bm25(query, data)
    
    result.sort(key=operator.itemgetter(1), reverse=True)
    
    for i in range(5):
        answer.append(result[i][0])

    
    return answer

<h2>For testing propose only</h2>

In [14]:
def get_google_result(feedback):
    result = []
    for doc in feedback.google:
        result.append(ast.literal_eval(doc))
    return result

def get_seach_result(feedback, model):
    '''
    Get a result from the feedback in order to compare with the implemented algorithm.
    
    ARGS:
        feedback: Feedback to compare with algoritm results.
        model: Model to get specific result.
    RESULT:
        Feedback for the given model.
    '''
    
    source = []
    result = []
    if (model == 'binary'):
        source = feedback.busca_binaria
    elif (model == 'tf'):
        source = feedback.tf
    elif (model == 'tf-idf'):
        source = feedback.tfidf
    else:
        source = feedback.bm25
    
    for i in range(len(source)):
        result.append(ast.literal_eval(source[i]))
    return result

def get_str_search(feedback, model, data):
    result = []
    for i in range(5):
        result.append(top_five(feedback.str_busca[i], model, data))
    return result


<h1>VSM BINARY PRECISION</h1>

In [87]:
result = get_str_search(feed_back, 'binary', df)
answer = get_seach_result(feed_back, 'binary')
google = get_google_result(feed_back)
   
print('Binary Precision: %.3f' % (mapk(answer, result, k=5)))
print('Precision Google: %.3f' % (mapk(google, result, k=5)))



Binary Precision: 0.240
Precision Google: 0.040


<h1>VSM TF PRECISION</h1>

In [88]:
tf_result = get_str_search(feed_back, 'tf', df)
answer = get_seach_result(feed_back, 'tf')
google = get_google_result(feed_back)

print('TF Precision: %.3f' % (mapk(answer, tf_result, k=5)))
print('Precision Google: %.3f' % (mapk(google, tf_result, k=5)))



TF Precision: 1.000
Precision Google: 0.048


<h1>VSM TF-IDF PRECISION</h1>

In [89]:
tf_idf_result = get_str_search(feed_back, 'tf-idf', df)
answer = get_seach_result(feed_back, 'tf-idf')
google = get_google_result(feed_back)

print('TF-IDF Precision: %.3f' % (mapk(answer, tf_idf_result, k=5)))
print('Precision Google: %.3f' % (mapk(google, tf_idf_result, k=5)))



TF-IDF Precision: 0.761
Precision Google: 0.084


<h1>VSM BM25 PRECISION</h1>

In [90]:
bm25_result = get_str_search(feed_back, 'bm25', df)
answer = get_seach_result(feed_back, 'bm25')
google = get_google_result(feed_back)

print('BM25 Precision: %.3f' % (mapk(answer, bm25_result, k=5)))
print('Precision Google: %.3f' % (mapk(google, bm25_result, k=5)))


BM25 Precision: 0.737
Precision Google: 0.118


<h1>Avaliation</h1>

<p>

For all result, the best output was by VSM TF witch gave a precision of 1.0, but the google precision was not that good with 0.048.

The best google precision was by VSM BM25 with 0.118, also showed a resonable result with 0.737.

</p>